In [ ]:
from my_flask import Flask, jsonify
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from datetime import datetime, timedelta


In [ ]:
# Create the Flask app
app = Flask(__name__)


In [ ]:
# Create the database connection
engine = create_engine('sqlite:///hawaii.sqlite')
Base = automap_base()
Base.prepare(engine, reflect=True)
Measurement = Base.classes.measurement
Station = Base.classes.station
session = Session(engine)


In [ ]:
# Define the routes

@app.route("/")
def home():
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/&lt;start&gt;<br/>"
        f"/api/v1.0/&lt;start&gt;/&lt;end&gt;<br/>"
    )

In [ ]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    # Calculate the date one year ago from the most recent date in the database
    last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
    one_year_ago = (datetime.strptime(last_date, '%Y-%m-%d') - timedelta(days=365)).strftime('%Y-%m-%d')

    
    # Query the precipitation data for the last 12 months
    results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_year_ago).all()
    
    # Convert the results to a dictionary with date as the key and prcp as the value
    precipitation_data = {date: prcp for date, prcp in results}
    
    return jsonify(precipitation_data)


In [ ]:
@app.route("/api/v1.0/stations")
def stations():
    # Query all the stations
    results = session.query(Station.station).all()
    
    # Convert the results to a list
    station_list = [station[0] for station in results]
    
    return jsonify(station_list)


In [ ]:
@app.route("/api/v1.0/tobs")
def tobs():
    # Calculate the date one year ago from the most recent date in the database
    last_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
    one_year_ago = (datetime.strptime(last_date, '%Y-%m-%d') - timedelta(days=365)).strftime('%Y-%m-%d')

    
    # Find the most-active station
    most_active_station = session.query(Measurement.station).\
        group_by(Measurement.station).\
        order_by(Measurement.station.desc()).\
        first()[0]
    
    # Query the temperature observations for the most-active station in the last 12 months
    results = session.query(Measurement.tobs).\
        filter(Measurement.station == most_active_station).\
        filter(Measurement.date >= one_year_ago).all()
    
    # Convert the results to a list
    tobs_list = [tobs[0] for tobs in results]
    
    return jsonify(tobs_list)

In [ ]:
@app.route("/api/v1.0/<start>")
@app.route("/api/v1.0/<start>/<end>")
def temperature_stats(start, end=None):
    # Query the temperature data based on the start and end dates (if provided)
    if end:
        results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
            filter(Measurement.date >= start).\
            filter(Measurement.date <= end).all()
    else:
        results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
            filter(Measurement.date >= start).all()
    
    # Convert the results to a dictionary
    temperature_stats = {
        'TMIN': results[0][0],
        'TAVG': results[0][1],
        'TMAX': results[0][2]
    }
    
    return jsonify(temperature_stats)

if __name__ == '__main__':
    app.run(debug=True)